In [ ]:
import pandas as pd

# Load the dataset

In [ ]:
df = pd.read_csv('data/adult19.csv')
pd.set_option('display.max_columns', 85)
pd.set_option('display.max_rows', 85)

# 1. Understand our data

### Head and Tail of the DataFrame: To get a quick look at the first and last few rows of your data.

In [ ]:
df.head()  # First 5 rows

In [ ]:
df.tail()  # Last 5 rows

### Frequency Distributions

In [ ]:
df['VISIONDF_A'].value_counts()

In [19]:
df['AGEP_A'].value_counts()

AGEP_A
85    1203
65     640
63     624
60     623
62     620
66     600
61     597
59     581
55     580
54     573
64     567
70     566
69     565
58     563
72     556
68     550
34     548
71     546
39     537
36     536
67     535
28     535
56     534
57     522
31     521
33     519
38     515
37     507
50     504
32     503
35     499
29     499
43     496
53     493
30     493
40     488
48     486
52     481
42     481
44     455
47     454
41     452
51     451
49     450
46     447
45     447
26     447
27     440
73     406
25     388
76     385
23     362
74     358
24     357
75     354
22     338
77     337
78     299
21     290
20     288
18     283
79     265
80     262
19     247
81     241
83     215
82     212
84     200
97      73
99       8
Name: count, dtype: int64

In [ ]:
pd.crosstab(df['VISIONDF_A'], columns='Frequency', margins=True)

In [ ]:
def frequency_table(df, column):
    # Frequency
    freq = df[column].value_counts(dropna=False).sort_index()

    # Percent
    percent = df[column].value_counts(normalize=True, dropna=False).sort_index() * 100

    # Valid Percent (excluding NaN)
    valid_percent = df[column].value_counts(normalize=True).sort_index() * 100

    # Cumulative Percent
    cum_percent = valid_percent.cumsum()

    # Combine all into a DataFrame
    freq_table = pd.DataFrame({'Frequency': freq, 'Percent': percent, 
                               'Valid Percent': valid_percent, 'Cumulative Percent': cum_percent})
    
    return freq_table

# List of columns to create frequency tables for
cols = ['VISIONDF_A', 'HEARINGDF_A', 'DIFF_A', 'COGMEMDFF_A', 'UPPSLFCR_A', 'COMDIFF_A']

# Create and display frequency tables for each column
for column in cols:
    print(f"Frequency Table for {column}:\n", frequency_table(df, column), "\n")


# 2. Handle Missing Values

In [ ]:
missing_values = [7, 8, 9]
df[cols] = df[cols].replace(missing_values, [pd.NA]*len(missing_values))

# Create and display frequency tables for each column
for column in cols:
    print(f"Frequency Table for {column}:\n", frequency_table(df, column), "\n")


# 3. Create Summation Variables

In [ ]:
# Define the conditions for SUM_234, where levels 2, 3, or 4 indicate some difficulty
sum_234_conditions = [(df[col] >= 2) & (df[col] <= 4) for col in cols]
df['SUM_234'] = sum(sum_234_conditions)

# For records missing all domain values, set SUM_234 to missing
df.loc[df[cols].isna().all(axis=1), 'SUM_234'] = pd.NA

# Calculate and display the frequency table for SUM_234
frequency_table(df, 'SUM_234')

In [ ]:
# Define the conditions for SUM_34, where levels 3 or 4 indicate a lot of difficulty or cannot do at all
sum_34_conditions = [(df[col] >= 3) & (df[col] <= 4) for col in cols]
df['SUM_34'] = sum(sum_34_conditions)

# For records missing all domain values, set SUM_34 to missing
df.loc[df[cols].isna().all(axis=1), 'SUM_34'] = pd.NA

# Calculate and display the frequency table for SUM_34
frequency_table(df, 'SUM_34')

# 4. Calculate Disability Identifiers

In [ ]:
df['DISABILITY1'] = (df['SUM_234'] >= 1).astype(int)
df['DISABILITY2'] = ((df['SUM_234'] >= 2) | (df['SUM_34'] >= 1)).astype(int)
df['DISABILITY3'] = df[cols].isin([3, 4]).any(axis=1).astype(int)
df['DISABILITY4'] = df[cols].isin([4]).any(axis=1).astype(int)

# Set missing values for Disability Identifiers
for col in ['DISABILITY1', 'DISABILITY2', 'DISABILITY3', 'DISABILITY4']:
    df.loc[df[cols].isna().all(axis=1), col] = pd.NA

dis_cols = ['DISABILITY1', 'DISABILITY2', 'DISABILITY3', 'DISABILITY4']
# Create and display frequency tables for each column
for column in dis_cols:
    print(f"Frequency Table for {column}:\n", frequency_table(df, column), "\n")

In [ ]:
df.head()